
# Sequence Modeling in NLP: Hidden Markov Models and Viterbi Decoding
## By Kevin Veeder

## Overview
This project explores the development and evaluation of a Hidden Markov Model (HMM) for part-of-speech tagging, combined with the Viterbi algorithm for sequence decoding. Leveraging the Brown Corpus from the NLTK library — specifically the news, editorial, and reviews categories — the model is trained and tested using the universal POS tagset.

The goal of this project is to implement a statistical sequence model from scratch, gain hands-on experience with foundational NLP techniques, and examine the strengths and limitations of HMM-based tagging in modern natural language processing pipelines.



## Preparing the Environment

To begin the project, I set up the NLP environment by importing key libraries. I used NLTK to access the Brown Corpus and its tagging tools, and `train_test_split` from `sklearn.model_selection` to prepare training and test data. The Brown Corpus—one of the first large-scale English corpora—offers 500 categorized text sources (e.g., news, editorial, reviews), making it ideal for sequence tagging tasks.

I downloaded both the corpus and the universal part-of-speech tagset using `nltk.download()`, ensuring all components were in place for training and evaluating the HMM and Viterbi models.



In [1]:
import nltk
from sklearn.model_selection import train_test_split
nltk.download('brown')
nltk.download('universal_tagset')

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\kevve\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\kevve\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True


## Loading and Exploring the Data

To begin the sequence tagging task, I loaded the Brown Corpus from NLTK, focusing on the news, editorial, and reviews categories to capture a mix of writing styles. I used the `tagged_sents()` method with the universal tagset, which simplifies part-of-speech categories into general types like `NOUN`, `VERB`, `ADJ`, `ADV`, and others—making the analysis more interpretable and language-agnostic.

The data is structured as a list of sentences, where each sentence is itself a list of (`word`, `tag`) pairs. This format is ideal for building and evaluating sequence models like HMMs.

Before moving into modeling, I explored the dataset to understand the tag distribution and sentence structure, which helped inform how I approached training and evaluation later in the project.

In [2]:
from nltk.corpus import brown
brown_sents = brown.tagged_sents(categories=['news', 'editorial', 'reviews'], tagset='universal')
brown_sents[0]

[('The', 'DET'),
 ('Fulton', 'NOUN'),
 ('County', 'NOUN'),
 ('Grand', 'ADJ'),
 ('Jury', 'NOUN'),
 ('said', 'VERB'),
 ('Friday', 'NOUN'),
 ('an', 'DET'),
 ('investigation', 'NOUN'),
 ('of', 'ADP'),
 ("Atlanta's", 'NOUN'),
 ('recent', 'ADJ'),
 ('primary', 'NOUN'),
 ('election', 'NOUN'),
 ('produced', 'VERB'),
 ('``', '.'),
 ('no', 'DET'),
 ('evidence', 'NOUN'),
 ("''", '.'),
 ('that', 'ADP'),
 ('any', 'DET'),
 ('irregularities', 'NOUN'),
 ('took', 'VERB'),
 ('place', 'NOUN'),
 ('.', '.')]


## Data Preprocessing

To prepare the data for training the Hidden Markov Model, I first applied a key preprocessing step: lowercasing all words in the dataset. This helps reduce vocabulary size and model complexity by treating words like “The” and “the” as the same token.

I preserved the original structure of the data—lists of (`word`, `tag`) pairs nested within sentences—ensuring compatibility with the rest of the pipeline.



In [3]:
brown_sents_lower = [[(word.lower(), tag) for word, tag in element]
                    for element in brown_sents]
brown_sents_lower[0]

[('the', 'DET'),
 ('fulton', 'NOUN'),
 ('county', 'NOUN'),
 ('grand', 'ADJ'),
 ('jury', 'NOUN'),
 ('said', 'VERB'),
 ('friday', 'NOUN'),
 ('an', 'DET'),
 ('investigation', 'NOUN'),
 ('of', 'ADP'),
 ("atlanta's", 'NOUN'),
 ('recent', 'ADJ'),
 ('primary', 'NOUN'),
 ('election', 'NOUN'),
 ('produced', 'VERB'),
 ('``', '.'),
 ('no', 'DET'),
 ('evidence', 'NOUN'),
 ("''", '.'),
 ('that', 'ADP'),
 ('any', 'DET'),
 ('irregularities', 'NOUN'),
 ('took', 'VERB'),
 ('place', 'NOUN'),
 ('.', '.')]

## Split Train and Test

Next, I split the preprocessed dataset into training and testing sets using `train_test_split` from `sklearn.model_selection`. This allowed for controlled evaluation of the model's performance. I specified the split ratio and optionally set a random_state to ensure reproducibility during development.



In [4]:
train, test =  train_test_split(brown_sents_lower, test_size = 0.20, random_state=42)


## Training the Hidden Markov Model with Viterbi

To support model training, I created a vocabulary list containing all unique words from the training set, as well as a separate list of all unique POS tags. These dictionaries serve as the foundation for estimating transition and emission probabilities in the HMM.

Importantly, I constructed these lists using only the training data to avoid data leakage—the risk of inadvertently exposing the model to information from the test set during training, which can inflate performance metrics and reduce generalizability.

In [5]:
unique_words = set()
for line in train:
    for word, tag in line:
        if word not in unique_words:
            unique_words.add(word)

unique_tags = set()
for line in train:
    for word, tag in line:
        if tag not in unique_tags:
            unique_tags.add(tag)
unique_tags

{'.',
 'ADJ',
 'ADP',
 'ADV',
 'CONJ',
 'DET',
 'NOUN',
 'NUM',
 'PRON',
 'PRT',
 'VERB',
 'X'}

I leveraged NLTK’s `HiddenMarkovModelTrainer` class, which integrates both the Hidden Markov Model and the Viterbi decoding algorithm. This combination enables the model to learn probabilities from training data and then infer the most likely sequence of tags for new sentences.

I initialized the trainer with the previously created tagset and vocabulary lists to set up the HMM training environment.

To explore the impact of different smoothing techniques on model performance, I trained five distinct HMM variants:

- Basic HMM without smoothing: Trained using raw counts to understand baseline performance.

- Lidstone smoothing (γ = 0.01): A slight smoothing to handle zero-frequency issues.

- Lidstone smoothing (γ = 0.1): Stronger smoothing to observe effects on generalization.

- Maximum Likelihood Estimation (MLE): Estimates parameters to maximize the probability of observed data.

- Expected Likelihood Estimation (ELE): Balances frequent and rare events through probabilistic adjustments.

Each model was trained using the `train_supervised()` method with the training dataset, applying the respective probability distribution functions where applicable. This multi-pronged approach allowed me to assess how smoothing choices influence tagging accuracy and robustness.

In [6]:
from nltk.probability import LidstoneProbDist, MLEProbDist, ELEProbDist

def lidstone_prob_dist_001(fd, bins):
    return LidstoneProbDist(fd, 0.01)

def lidstone_prob_dist_01(fd, bins):
    return LidstoneProbDist(fd, 0.1)

def MLE_ProbDist(fd, bins):
    return MLEProbDist(fd)

def ELE_ProbDist(fd, bins):
    return ELEProbDist(fd)

In [7]:
# Importing necessary modules from NLTK for HMM training and probability distributions
from nltk.tag import HiddenMarkovModelTrainer
from nltk.probability import LidstoneProbDist, MLEProbDist, ELEProbDist

trainer = HiddenMarkovModelTrainer(unique_tags, unique_words)

mod1 = trainer.train_supervised(train)

mod2 = trainer.train_supervised(train, lidstone_prob_dist_001)

mod3 = trainer.train_supervised(train, lidstone_prob_dist_01)

mod4 = trainer.train_supervised(train, MLE_ProbDist)

mod5 = trainer.train_supervised(train, ELE_ProbDist)

## Applying the HMM with Viterbi Algorithm

With the five trained HMM models ready, I proceeded to predict part-of-speech tags on the test dataset to evaluate their performance.

For each model, I used the `best_path()` method, which implements the Viterbi algorithm to find the most likely sequence of tags for a given sentence. Since `best_path()` requires untagged input, I extracted only the words from each sentence in the test set—ignoring the true tags during prediction.

This step allowed me to compare how each smoothing method influenced the accuracy of tag predictions on unseen data.

In [8]:
new_test = test.copy()
broken_test = []
for line in new_test:
    broken_line = []
    for word, tag in line:
        broken_line.append(word)
    broken_test.append(broken_line)

    
broken_test


[['conservation', 'plan'],
 ['pirate',
  'manager',
  'danny',
  'murtaugh',
  'said',
  'he',
  "hadn't",
  'decided',
  'between',
  'mizell',
  'and',
  'vern',
  'law',
  'for',
  "wednesday's",
  'game',
  '.'],
 ['it', 'tends', 'to', 'be', 'hard', 'and', 'colorless', '.'],
 ['agriculture',
  'department',
  'economists',
  'estimate',
  'the',
  'government',
  'this',
  'year',
  'will',
  'hand',
  'farmers',
  '$1.4',
  'billion',
  'in',
  'special',
  'subsidies',
  'and',
  'incentive',
  'payments',
  ',',
  'well',
  'above',
  'the',
  'record',
  '$1.1',
  'billion',
  'of',
  '1958',
  'and',
  'about',
  'double',
  'the',
  '$639',
  'million',
  'of',
  '1960',
  '.'],
 ['deadlock', 'on', 'tests'],
 ["keys's",
  'findings',
  ',',
  'though',
  'far',
  'from',
  'complete',
  ',',
  'are',
  'likely',
  'to',
  'smash',
  'many',
  'an',
  'eating',
  'cliche',
  '.'],
 ['the',
  'primitive-eclogue',
  'quality',
  'of',
  'his',
  'drawings',
  ',',
  'akin',
  't

In [9]:
predictions_mod1 = []
for line in broken_test:
    preds = mod1.best_path(line)
    predictions_mod1.append(preds)
    
predictions_mod2 = []
for line in broken_test:
    preds = mod2.best_path(line)
    predictions_mod2.append(preds)

predictions_mod3 = []
for line in broken_test:
    preds = mod3.best_path(line)
    predictions_mod3.append(preds)

predictions_mod4 = []
for line in broken_test:
    preds = mod4.best_path(line)
    predictions_mod4.append(preds)

predictions_mod5 = []
for line in broken_test:
    preds = mod5.best_path(line)
    predictions_mod5.append(preds)


C:\Users\kevve\anaconda3\Lib\site-packages\nltk\tag\hmm.py:336: RuntimeWarning: overflow encountered in cast
  O[i, k] = self._output_logprob(si, self._symbols[k])
C:\Users\kevve\anaconda3\Lib\site-packages\nltk\tag\hmm.py:334: RuntimeWarning: overflow encountered in cast
  X[i, j] = self._transitions[si].logprob(self._states[j])
C:\Users\kevve\anaconda3\Lib\site-packages\nltk\tag\hmm.py:364: RuntimeWarning: overflow encountered in cast
  O[i, k] = self._output_logprob(si, self._symbols[k])


In [10]:
list(zip(broken_test[1], predictions_mod1[1]))

[('pirate', 'NOUN'),
 ('manager', 'NOUN'),
 ('danny', 'NOUN'),
 ('murtaugh', 'NOUN'),
 ('said', 'VERB'),
 ('he', 'PRON'),
 ("hadn't", 'VERB'),
 ('decided', 'VERB'),
 ('between', 'ADP'),
 ('mizell', 'NOUN'),
 ('and', 'CONJ'),
 ('vern', 'DET'),
 ('law', 'DET'),
 ('for', 'DET'),
 ("wednesday's", 'DET'),
 ('game', 'DET'),
 ('.', 'DET')]

In [11]:
labels_predicted_mod1 = []
for i in range(len(broken_test)):
    new_list = list(zip(broken_test[i], predictions_mod1[i]))
    labels_predicted_mod1.append(new_list)

labels_predicted_mod2 = []
for i in range(len(broken_test)):
    new_list = list(zip(broken_test[i], predictions_mod2[i]))
    labels_predicted_mod2.append(new_list)

labels_predicted_mod3 = []
for i in range(len(broken_test)):
    new_list = list(zip(broken_test[i], predictions_mod3[i]))
    labels_predicted_mod3.append(new_list)

labels_predicted_mod4 = []
for i in range(len(broken_test)):
    new_list = list(zip(broken_test[i], predictions_mod4[i]))
    labels_predicted_mod4.append(new_list)

labels_predicted_mod5 = []
for i in range(len(broken_test)):
    new_list = list(zip(broken_test[i], predictions_mod5[i]))
    labels_predicted_mod5.append(new_list)


## Model Evaluation

Evaluating the performance of each HMM variant was a crucial part of this project to understand how well the models generalize to unseen data. I compared the predicted tags against the true tags from the test set using two key evaluation tools:

- Precision, Recall, and F1-score: Computed via the provided `printConlleval` function, these metrics offer detailed insight into per-tag and overall model accuracy.

- Confusion Matrix: Generated with the `printConfusionMatrix` function to visualize common misclassifications and error patterns.

Both functions required predictions and ground truths formatted as lists of sentences, where each sentence is a list of (`word`, `tag`) pairs.

After running the evaluations for each model, here are my key observations:

- The Lidstone-smoothed HMM with gamma = 0.01 (mod2) demonstrated the best overall performance.

- The difference between Lidstone smoothing with gamma = 0.01 and 0.1 was minor, though gamma = 0.01 performed slightly better.

- The ‘pure’ HMM without smoothing lagged behind, correctly tagging only about half of the tokens.

- The ‘pure’ HMM and the MLE-based model showed similar performance, suggesting they rely on closely related algorithms.

- Models 1 and 4 performed best when tagging the ‘X’ category, which covers undefined or rare cases.

- No additional models beyond the five described were trained.

This evaluation not only quantified model accuracy but also provided a basis for discussing smoothing effects and model robustness. I encouraged further discussion with peers and teaching assistants to deepen understanding, while keeping specific implementations confidential.


In [12]:
from nltk.metrics import ConfusionMatrix
import itertools

def printConfusionMatrix(labels_predicted, labels_correct):    
    actual_tags = list(itertools.chain(*[[tag for word, tag in sent] for sent in labels_correct]))
    predicted_tags = list(itertools.chain(*[[tag for word, tag in sent] for sent in labels_predicted]))
    conf_matrix = ConfusionMatrix(actual_tags, predicted_tags)    
    print(conf_matrix)


In [13]:
# Importing necessary libraries for model evaluation and metrics calculation
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelBinarizer
from itertools import chain
import numpy as np

# Defining the conlleval function for evaluating NLP models
def printConlleval(labels_predicted, labels_correct):
    lb = LabelBinarizer() # Initializing the LabelBinarizer for handling label encoding

    # Flattening the list of labels for correct and predicted
    labels_correct_flattened = [(word, tag) for sent in labels_correct for word, tag in sent]
    labels_predicted_flattened = [(word, tag) for sent in labels_predicted for word, tag in list(sent)]

    # Transforming the labels into a binary format for evaluation
    y_true_combined = lb.fit_transform([tag for _, tag in labels_correct_flattened])
    y_pred_combined = lb.transform([tag for _, tag in labels_predicted_flattened])

    tagset = set(lb.classes_)
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}

    num_sentences = len(labels_predicted)
    total_tokens = sum(len(s) for s in labels_predicted)
     
    num_correct_sentences, total_correct_tokens = 0, 0
    for pred, true in zip(labels_predicted, labels_correct):
        if len(pred) == len(true):  
            correct_tokens = sum(p == t for p, t in zip(pred, true))
            total_correct_tokens += correct_tokens
            if correct_tokens == len(pred):
                num_correct_sentences += 1

    correct_sentences_percentage = num_correct_sentences / num_sentences * 100
    total_correct_tokens_percentage = total_correct_tokens / total_tokens * 100

    classification_report_dict = classification_report(
        y_true_combined,
        y_pred_combined,
        labels=[class_indices[cls] for cls in tagset],
        target_names=tagset,
        output_dict=True,
        zero_division=1
    )

 
    classification_report_dict.pop('macro avg', None)
    classification_report_dict.pop('weighted avg', None)
    classification_report_dict.pop('samples avg', None)
    classification_report_dict.pop('micro avg', None)

    total_precision = precision_score(y_true_combined, y_pred_combined, average='weighted', zero_division=1)
    total_recall = recall_score(y_true_combined, y_pred_combined, average='weighted', zero_division=1)
    total_f1 = f1_score(y_true_combined, y_pred_combined, average='weighted', zero_division=1)
    total_line = f"{'Total':<15s} {total_precision:<10.2f} {total_recall:<10.2f} {total_f1:<10.2f}"

    report_lines = [f"{k:<15s} {classification_report_dict[k]['precision']:<10.2f} {classification_report_dict[k]['recall']:<10.2f} {classification_report_dict[k]['f1-score']:<10.2f}" for k in classification_report_dict if isinstance(classification_report_dict[k], dict)]
    report_lines.insert(0, "\n")
    report_lines.insert(1, f"{'TAG':<15s} {'Precision':<10s} {'Recall':<10s} {'F1-score':<10s}\n")
    report_lines.insert(2, total_line)
    report_lines.insert(3, '-'*50 + '\n') 
    classification_report_str = "\n".join(report_lines)

    additional_info_str = ''
    additional_info_str += f'Total tokens: {total_tokens}\n'
    additional_info_str += f'Total correct tokens: {total_correct_tokens} ({total_correct_tokens_percentage:.2f}%)\n'
    additional_info_str += f'Processed sentences: {num_sentences}\n'
    additional_info_str += f'Completely correct sentences: {num_correct_sentences} ({correct_sentences_percentage:.2f}%)\n'

    print(additional_info_str + classification_report_str)

In [14]:
conf_matrix_mod1 = printConfusionMatrix(labels_predicted_mod1, test)
conf_matrix_mod2 = printConfusionMatrix(labels_predicted_mod2, test)
conf_matrix_mod3 = printConfusionMatrix(labels_predicted_mod3, test)
conf_matrix_mod4 = printConfusionMatrix(labels_predicted_mod4, test)
conf_matrix_mod5 = printConfusionMatrix(labels_predicted_mod5, test)

     |                        C         N         P         V      |
     |         A    A    A    O    D    O    N    R    P    E      |
     |         D    D    D    N    E    U    U    O    R    R      |
     |    .    J    P    V    J    T    N    M    N    T    B    X |
-----+-------------------------------------------------------------+
   . |<2152>   .    .    .    . 2659    .    .    .    .    .    . |
 ADJ |    .<1429>   .   69    . 1545   38    .    .    5    5    . |
 ADP |    .    .<2586>  29    4 2307    .    .   10   29    4    . |
 ADV |    .   38   36 <947>   .  711    1    .    .   13    1    . |
CONJ |    .    .    .    . <561> 608    .    .    .    .    .    . |
 DET |    .    .   14    3    .<4633>   .    .   22    .    .    . |
NOUN |    1   51    1    2    . 5682<5245>   8    1    .   62    . |
 NUM |    .    .    .    .    .  316    2 <364>   .    .    .    . |
PRON |    .    .    6    .    .  420    1    . <848>   .    .    . |
 PRT |    .    1   55    3    .  3

In [15]:
conlleval_mod1 = printConlleval(labels_predicted_mod1, test)
conlleval_mod2 = printConlleval(labels_predicted_mod2, test)
conlleval_mod3 = printConlleval(labels_predicted_mod3, test)
conlleval_mod4 = printConlleval(labels_predicted_mod4, test)
conlleval_mod5 = printConlleval(labels_predicted_mod5, test)

Total tokens: 40434
Total correct tokens: 22590 (55.87%)
Processed sentences: 1875
Completely correct sentences: 489 (26.08%)


TAG             Precision  Recall     F1-score  

Total           0.88       0.56       0.62      
--------------------------------------------------

.               1.00       0.45       0.62      
ADJ             0.94       0.46       0.62      
ADP             0.96       0.52       0.67      
ADV             0.90       0.54       0.68      
CONJ            0.99       0.48       0.65      
DET             0.21       0.99       0.35      
NOUN            0.97       0.47       0.64      
NUM             0.98       0.53       0.69      
PRON            0.96       0.67       0.79      
PRT             0.91       0.52       0.66      
VERB            0.98       0.56       0.71      
X               1.00       0.28       0.43      
Total tokens: 40434
Total correct tokens: 37268 (92.17%)
Processed sentences: 1875
Completely correct sentences: 585 (31.20%)


TAG  

## Choosing best model

After evaluating all models, I selected the best-performing one (Lidstone smoothing with gamma = 0.01) and exported it using the `dill` library for easy reuse and deployment. The model was saved as `mybestmodel.dill` in the project directory to maintain consistency for submission and future loading.

This serialized file preserves the trained model state, enabling quick loading without retraining, which is essential for practical applications and sharing.

In [16]:
# Importing dill library for model serialization
import dill
mybestmodel = mod2

# serialization with dill
with open('mybestmodel.dill', 'wb') as file:
    dill.dump(mybestmodel, file)   